In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
from rl_env_inc_skip import TrajComp
from rl_brain import PolicyGradient
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

def run_online(env, RL, ratio, elist): # Validation
    eva = []
    skip_pts = 0
    step_pts = 0
    for episode in elist:
        buffer_size = int(ratio*len(env.ori_traj_set[episode]))
        if buffer_size < 3:
            continue
        steps, observation = env.reset(episode, buffer_size)
        step_pts = step_pts + steps
        for index in range(buffer_size, steps):
            if index == steps - 1:
                done = True
            else:
                done = False
            if index < env.INX:
                #print('test skip')
                skip_pts = skip_pts + 1
                continue
            action = RL.pro_choose_action(observation)
            observation_, _ = env.step(episode, action, index, done, 'V') #'T' means Training, and 'V' means Validation
            observation = observation_
        eva.append(env.output(episode, 'V')) #'T' means Training, 'V' means Validation, and 'V-VIS' for visualization on Validation
    return eva
        
def run_comp(env, RL, Round, traj_amount, valid_amount, show_time, ratio): #Training
    check = 999999
    tra_av_errs = []#放的是每50回合训练集上的平均误差
    val_av_errs = []#放的是每50回合验证集上的平均误差
    for r in range(Round):
        env.shuffle()
        episode = 0
        for i in range(0, traj_amount, show_time):
            train_ep_errs = []
            start_t = time.time()
            for _ in tqdm(range(show_time), desc="[{}/{}]".format(min(i+show_time,traj_amount),traj_amount),ncols=100):
                buffer_size = int(ratio*len(env.ori_traj_set[episode]))
                # extreme cases
                if buffer_size < 3:
                    episode += 1
                    continue
                steps, state = env.reset(episode, buffer_size)#初始化状态值，返回轨迹点数和有序列表前k个状态值
                for index in range(buffer_size, steps):#从第一次缓存外第一个点遍历到最后一个点
                    if index == steps - 1:#如果已经是轨迹最后一个点
                        done = True
                    else:
                        done = False
                    if index < env.INX:
                        #print('train skip')
                        continue
                    action = RL.pro_choose_action(state)#状态输出到神经网络输出动作的概率分布，按概率采样一个动作
                    new_state, reward = env.step(episode, action, index, done, 'T') #'T' means Training, and 'V' means Validation                   
                    RL.store_transition(state, action, reward)
                    if done:
                        RL.learn()#回合结束开始策略梯度算法学习参数
                        break
                    state = new_state
                train_e = env.output(episode, 'T') #'T' means Training, 'V' means Validation, and 'V-VIS' for visualization on Validation
                train_ep_errs.append(train_e)
                episode += 1
            
            val_ep_errs = run_online(env, RL, ratio, [i for i in range(traj_amount, traj_amount + valid_amount)])
            val_av_err = sum(val_ep_errs)/len(val_ep_errs)
            val_av_errs.append(val_av_err)
            tra_av_err = sum(train_ep_errs)/len(train_ep_errs)
            tra_av_errs.append(tra_av_err)
            print('round {} episode {}: Training error: {}, Validation error: {}'.format(r, episode, tra_av_err, val_av_err))
            if val_av_err < check:
                check = val_av_err
                RL.save('./save_skip_multitrain/'+ str(val_av_err) + '_ratio_' + str(ratio) + '_' + env.label + '/trained_model.ckpt')
                print('Save model with error {}'.format(val_av_err))
            print('==>current best model is {} with ratio {}'.format(check, ratio))
            print('It costs {}s'.format(time.time()-start_t))
    return tra_av_errs, val_av_errs

def train(traj_amount, valid_amount, Round, show_time, ratio, a_size, s_size, skip_size, label):
    traj_path = '../trajData/Geolife_out/'
    env = TrajComp(a_size + skip_size, s_size)
    env.load_train_data(traj_path, traj_amount, valid_amount)
    env.set_error_type(label)
    RL = PolicyGradient(env.n_features, env.n_actions)
    start = time.time()
    tra_av_errs, val_av_errs = run_comp(env, RL, Round, traj_amount, valid_amount, show_time, ratio)
    print("Training elapsed time = %s", float(time.time() - start))
    with open('errors_records.txt', 'a') as f:
        f.write('\nTraining errors and validation errors (' + label + ')\n')
        for i in range(len(tra_av_errs)):
            f.write(str(tra_av_errs[i])+' ')
        f.write('\n')
        for i in range(len(val_av_errs)):
            f.write(str(val_av_errs[i])+' ')
    plt.figure()
    plt.title("Average errors of training and validation:")
    plt.xlabel("training process / "+str(show_time)+" episodes")
    plt.ylabel(label+" error")
    x = range(len(tra_av_errs))
    plt.plot(x, tra_av_errs, "r", label="training" )
    plt.plot(x, val_av_errs, "b", label="validation")
    plt.legend()
    plt.figure()
    plt.title("Average errors of validation:")
    plt.xlabel("training process / "+str(show_time)+" episodes")
    plt.ylabel(label+" error")
    x = range(len(tra_av_errs))
    plt.plot(x, val_av_errs, "b", label="validation")
    plt.legend()
    plt.show()

def retrain(model_path, traj_amount, valid_amount, Round, show_time, ratio, a_size, s_size, skip_size, label):
    traj_path = '../trajData/Geolife_out/'
    env = TrajComp(a_size + skip_size, s_size)
    env.load_train_data(traj_path, traj_amount, valid_amount)
    env.set_error_type(label)
    RL = PolicyGradient(env.n_features, env.n_actions)
    RL.load(model_path)
    start = time.time()
    tra_av_errs, val_av_errs = run_comp(env, RL, Round, traj_amount, valid_amount, show_time, ratio)
    print("Training elapsed time = %s", float(time.time() - start))
    with open('errors_records.txt', 'a') as f:
        f.write('\nTraining errors and validation errors (' + label + ')\n')
        for i in range(len(tra_av_errs)):
            f.write(str(tra_av_errs[i])+' ')
        f.write('\n')
        for i in range(len(val_av_errs)):
            f.write(str(val_av_errs[i])+' ')
    plt.figure()
    plt.title("Average errors of training and validation:")
    plt.xlabel("training process / "+str(show_time)+" episodes")
    plt.ylabel(label+" error")
    x = range(len(tra_av_errs))
    plt.plot(x, tra_av_errs, "r", label="training" )
    plt.plot(x, val_av_errs, "b", label="validation")
    plt.legend()
    plt.figure()
    plt.title("Average errors of validation:")
    plt.xlabel("training process / "+str(show_time)+" episodes")
    plt.ylabel(label+" error")
    x = range(len(tra_av_errs))
    plt.plot(x, val_av_errs, "b", label="validation")
    plt.legend()
    plt.show()
    
from rl_env_inc_skip import TrajComp
from rl_brain import PolicyGradient
import data_utils as F
import time

def evaluate(env, RL, ratio, elist): # Evaluation
    eva = []
    skip_pts = 0
    step_pts = 0
    for episode in elist:
        buffer_size = int(ratio*len(env.ori_traj_set[episode]))
        if buffer_size < 3:
            continue
        steps, observation = env.reset(episode, buffer_size)
        step_pts = step_pts + steps
        for index in range(buffer_size, steps):
            if index == steps - 1:
                done = True
            else:
                done = False
            if index < env.INX:
                #print('test skip')
                skip_pts = skip_pts + 1
                continue
            action = RL.quick_time_action(observation)
            observation_, _ = env.step(episode, action, index, done, 'V') #'T' means Training, and 'V' means Validation
            observation = observation_
        eva.append(env.output(episode, 'V')) #'T' means Training, 'V' means Validation, and 'V-VIS' for visualization on Validation
    return sum(eva)/len(eva)

def test(model_path, amount, ratio, a_size, s_size, skip_size, label):
    print("======Start testing the model at'{}======'".format(model_path))
    traj_path = '../trajData/Geolife_out/'
    env = TrajComp(a_size + skip_size, s_size)
    env.load_test_data(traj_path, amount)
    env.set_error_type(label)
    rl = PolicyGradient(env.n_features, env.n_actions)
    rl.load(model_path) #your_trained_model your_trained_model_skip
    for _ in range(3):
        start = time.time()
        effectiveness = evaluate(env, rl, ratio, range(amount))
        print("Effectiveness: {:.4f}".format(effectiveness))
        print("Training elapsed time = {:.4f}s".format(float(time.time() - start)))   

def test_err(model_path, amount, ratio, a_size, s_size, skip_size):
    print("======Start testing the model at'{}======'".format(model_path))
    traj_path = '../trajData/Geolife_out/'
    env = TrajComp(a_size + skip_size, s_size)
    env.load_test_data(traj_path, amount)
    rl = PolicyGradient(env.n_features, env.n_actions)
    rl.load(model_path) #your_trained_model your_trained_model_skip
    for _ in range(3):
        errs = []
        t = []
        for type in ['sed','ped','dad','sad']:
            env.set_error_type(type)
            st = time.time()
            err = evaluate(env, rl, ratio, range(amount))
            t.append(time.time()-st)
            errs.append(err)
        print("Effectiveness of different errors: sed:{:.4f}, ped:{:.4f}, dad:{:.4f}, sad:{:.4f}"\
              .format(errs[0],errs[1],errs[2],errs[3]))
        print("Testing elapsed time = {:.4f}s, {:.4f}s, {:.4f}s, {:.4f}s".format(t[0],t[1],t[2],t[3]))
        
def test_ratio(model_path, amount, a_size, s_size, skip_size, label):
    print("======Start testing the model at'{}======'".format(model_path))
    traj_path = '../trajData/Geolife_out/'
    env = TrajComp(a_size + skip_size, s_size)
    env.load_test_data(traj_path, amount)
    rl = PolicyGradient(env.n_features, env.n_actions)
    rl.load(model_path) #your_trained_model your_trained_model_skip
    for _ in range(3):
        errs = []
        t = []
        for r in [0.1,0.2,0.3,0.4,0.5]:
            env.set_error_type(label)
            st = time.time()
            err = evaluate(env, rl, r, range(amount))
            t.append(time.time()-st)
            errs.append(err)
        print("Effectiveness of different ratios: 0.1:{:.4f}, 0.2:{:.4f}, 0.3:{:.4f}, 0.4:{:.4f}, 0.5:{:.4f}"\
              .format(errs[0],errs[1],errs[2],errs[3],errs[4]))
        print("Testing elapsed time = {:.4f}s, {:.4f}s, {:.4f}s, {:.4f}s, {:.4f}s".format(t[0],t[1],t[2],t[3],t[4]))

def test_one_sample(model_path, index, ratio, a_size, s_size, skip_size, label):
    print("======Start testing the model at'{}======'".format(model_path))
    traj_path = '../trajData/Geolife_out/'
    env = TrajComp(a_size + skip_size, s_size)
    env.load_one_sample(traj_path, index)
    env.set_error_type(label)
    rl = PolicyGradient(env.n_features, env.n_actions)
    rl.load(model_path) #your_trained_model your_trained_model_skip
    
    net_time = 0
    step_time = 0
    start_ = time.time()
    eva = []
    buffer_size = int(ratio*len(env.ori_traj_set[0]))
    if buffer_size < 3:
        return
    steps, observation = env.reset(0, buffer_size)
    for index in range(buffer_size, steps):
        if index == steps - 1:
            done = True
        else:
            done = False
        start = time.time()
        action = rl.quick_time_action(observation)
        net_time += float(time.time()-start)
        start = time.time()
        observation_, _ = env.step(0, action, index, done, 'V') #'T' means Training, and 'V' means Validation
        step_time += float(time.time()-start)
        observation = observation_
    eva=env.output(0, 'V') #'T' means Training, 'V' means Validation, and 'V-VIS' for visualization on Validation
    print("Effectiveness: %e" %eva)
    print("Training elapsed time = %s", float(time.time() - start_))
    print("net_time:",net_time)
    print("step_time:",step_time)   

In [3]:
def multitrain(ratio, label):
    train(traj_amount=200, valid_amount=70, Round=3, show_time=100, ratio=ratio, a_size=3, s_size=3, skip_size=2, label=label)

In [7]:
from joblib import Parallel, delayed
t = time.time()
parallel = Parallel(n_jobs=-1, backend='loky', timeout=7200)
err_type = ['sed','ped','dad','sad','sed','ped','dad','sad']
ratio_type = [0.1,0.1,0.1,0.1,0.2,0.2,0.2,0.2]
out = parallel(delayed(multitrain)(ratio, label) for ratio, label in zip(ratio_type, err_type))
print('It cost {}s'.format(time.time()-t))

It cost 1390.849925994873s
